In [15]:
import pickle
from datetime import datetime
import pandas as pd

In [3]:
# Load the pre-trained model from a pickle file
with open('CrudeOilPricing.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [4]:
loaded_model

In [50]:
df=pd.read_csv(r'C:\Users\USER\Guvi pytho\Forage\Nat_Gas.csv')
df['Dates']=df['Dates'].str.replace('/','-')
df['Dates'] = pd.to_datetime(df['Dates'],format='%m-%d-%y').dt.date
df

,Dates,Prices
0,2020-10-31,10.10
1,2020-11-30,10.30
2,2020-12-31,11.00
3,2021-01-31,10.90
4,2021-02-28,10.90
5,2021-03-31,10.90
6,2021-04-30,10.40
7,2021-05-31,9.84
8,2021-06-30,10.00
9,2021-07-31,10.10


In [94]:
# The input parameters that should be taken into account for pricing are:
# Injection dates. 
# Withdrawal dates.
# The prices at which the commodity can be purchased/sold on those dates.
# The rate at which the gas can be injected/withdrawn.
# The maximum volume that can be stored.
# Storage costs.

injection_days =[datetime.strptime(input(f'Please enter the {i}th date in MM-DD-YYYY format'),"%m-%d-%Y").date() 
                 for i in range(1,(int(input('How many days have you planned to inject')))+1)]
withdrawal_days =[datetime.strptime(input(f'Please enter the {i}th date in MM-DD-YYYY format'),"%m-%d-%Y").date() 
                  for i in range(1,(int(input('How many days have you planned to withdraw')))+1)]
inject_withdraw_rate= int(input(f'Please enter the rate at which the gas can be injected/withdrawn :')) #MMBt
Max_storage=int(input(f'Please enter the maximum storage capacity :'))
storage_cost=int(input(f'Please enter the storage cost per period(month) :'))

How many days have you planned to inject 3
Please enter the 1th date in MM-DD-YYYY format 05-31-2022
Please enter the 2th date in MM-DD-YYYY format 05-31-2020
Please enter the 3th date in MM-DD-YYYY format 06-30-2018
How many days have you planned to withdraw 1
Please enter the 1th date in MM-DD-YYYY format 06-30-2025
Please enter the rate at which the gas can be injected/withdrawn : 1000
Please enter the maximum storage capacity : 100000000
Please enter the storage cost per period : 10


In [51]:
df['Dates'].values

array([datetime.date(2020, 10, 31), datetime.date(2020, 11, 30),
       datetime.date(2020, 12, 31), datetime.date(2021, 1, 31),
       datetime.date(2021, 2, 28), datetime.date(2021, 3, 31),
       datetime.date(2021, 4, 30), datetime.date(2021, 5, 31),
       datetime.date(2021, 6, 30), datetime.date(2021, 7, 31),
       datetime.date(2021, 8, 31), datetime.date(2021, 9, 30),
       datetime.date(2021, 10, 31), datetime.date(2021, 11, 30),
       datetime.date(2021, 12, 31), datetime.date(2022, 1, 31),
       datetime.date(2022, 2, 28), datetime.date(2022, 3, 31),
       datetime.date(2022, 4, 30), datetime.date(2022, 5, 31),
       datetime.date(2022, 6, 30), datetime.date(2022, 7, 31),
       datetime.date(2022, 8, 31), datetime.date(2022, 9, 30),
       datetime.date(2022, 10, 31), datetime.date(2022, 11, 30),
       datetime.date(2022, 12, 31), datetime.date(2023, 1, 31),
       datetime.date(2023, 2, 28), datetime.date(2023, 3, 31),
       datetime.date(2023, 4, 30), datetime.da

In [97]:
injection_days.sort()
withdrawal_days.sort()

[datetime.date(2018, 6, 30),
 datetime.date(2020, 5, 31),
 datetime.date(2022, 5, 31)]

In [99]:
all_days_df=pd.DataFrame(injection_days+withdrawal_days,columns=['Dates'])
all_days_df['Dates'] = pd.to_datetime(all_days_df['Dates'],format='%m-%d-%y').dt.date

def get_price(row):
    if row['Dates'] in df['Dates'].values:
        # Date exists in known data -> get that price
        print(df.loc[df['Dates'] == row['Dates'], 'Prices'])
        return df.loc[df['Dates'] == row['Dates'], 'Prices'].values[0]
    else:
        # Date not in known data -> predict
        return loaded_model.predict(pd.DataFrame({'ds': [row['Dates']]}))['yhat'].iloc[0]

# Apply the function to every row
all_days_df['Prices'] = all_days_df.apply(get_price, axis=1)


all_days_df=all_days_df.sort_values('Dates')
all_days_df 
 

19    10.7
Name: Prices, dtype: float64


,Dates,Prices
0,2018-06-30,8.315417
1,2020-05-31,9.276780
2,2022-05-31,10.700000
3,2025-06-30,11.848579


In [100]:
all_days_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Dates   4 non-null      object 
 1   Prices  4 non-null      float64
dtypes: float64(1), object(1)
memory usage: 196.0+ bytes


In [104]:
def price_storage_contract(
    injection_dates, withdrawal_dates, all_days_df,inject_withdraw_rate,
    max_storage, storage_cost) :

    total_inject_volume=0
    total_withdraw_volumne=0
    contract_value = 0.0
    inventory=0

    for date in all_days_df['Dates']:
        market_price = all_days_df.loc[all_days_df['Dates'] == date, 'Prices'].values[0]
        print(f'Market price on {date} is ${round(market_price,2)}')
        if date in injection_dates:
            inject_volume = min(inject_withdraw_rate, max_storage - inventory)
            inventory += inject_volume
            total_inject_volume+=inject_volume
            contract_value -= inject_volume * market_price 
        
        if date in withdrawal_dates:
            withdraw_volumne = min(inject_withdraw_rate, inventory)
            inventory -= withdraw_volumne
            total_withdraw_volumne+=withdraw_volumne
            contract_value += withdraw_volumne * market_price 
    # Calculate storage costs
    # For simplicity, assume storage costs are calculated based on the average volume stored
    average_volume = (total_inject_volume - total_withdraw_volumne) / 2
    start_date = min(withdrawal_dates[0], injection_dates[0])
    end_date = max(withdrawal_dates[-1], injection_dates[-1])
    month_difference = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
    contract_value -= average_volume * storage_cost * month_difference

    return contract_value
    

In [107]:
print(f'The estimated contract value is ${round(price_storage_contract(injection_days, withdrawal_days, all_days_df,inject_withdraw_rate,
    Max_storage, storage_cost),2)}M ')

Market price on 2018-06-30 is $8.32
Market price on 2020-05-31 is $9.28
Market price on 2022-05-31 is $10.7
Market price on 2025-06-30 is $11.85
The estimated contract value is $-856443.62M 
